A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [18]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /Users/dylanskinner/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [19]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed
set_seed(3407)

In [20]:
import pickle

class SortDataset(Dataset):
    """ 
    Dataset for the Sort problem. E.g. for problem length 6:
    Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
    Which will feed into the transformer concatenated as:
    input:  0 0 2 1 0 1 0 0 0 1 1
    output: I I I I I 0 0 0 1 1 2
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=10, num_digits=7):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
        self.num_digits = num_digits
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return self.num_digits
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return self.length * 2 - 1

    def __getitem__(self, idx):
        
        # use rejection sampling to generate an input example from the desired split
        while True:
            # generate some random integers
            inp = torch.randint(self.num_digits, size=(self.length,), dtype=torch.long)
            # half of the time let's try to boost the number of examples that 
            # have a large number of repeats, as this is what the model seems to struggle
            # with later in training, and they are kind of rate
            if torch.rand(1).item() < 0.5:
                if inp.unique().nelement() > self.length // 2:
                    # too many unqiue digits, re-sample
                    continue
            # figure out if this generated example is train or test based on its hash
            h = hash(pickle.dumps(inp.tolist()))
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        # solve the task: i.e. sort
        sol = torch.sort(inp)[0]

        # concatenate the problem specification and the solution
        cat = torch.cat((inp, sol), dim=0)

        # the inputs to the transformer will be the offset sequence
        x = cat[:-1].clone()
        y = cat[1:].clone()
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:self.length-1] = -1
        return x, y


In [21]:
# print an example instance of the dataset
train_dataset = SortDataset('train')
test_dataset = SortDataset('test')
x, y = train_dataset[0]
for a, b in zip(x,y):
    print(int(a),int(b))

0 -1
3 -1
3 -1
2 -1
0 -1
1 -1
0 -1
6 -1
3 -1
1 0
0 0
0 0
0 1
1 1
1 2
2 3
3 3
3 3
3 6


In [22]:
# create a GPT instance
from mingpt.model import GPT
import os

checkpoints = os.listdir('./checkpoints')
checkpoints.sort()


model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model_config.checkpoint = 'checkpoints/' + checkpoints[-1] if checkpoints else None  # This is a change
model = GPT(model_config)

number of parameters: 0.09M


In [23]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 1000 + model.iter_num if model_config.checkpoint else 1000  # This is a change
train_config.num_workers = 0
train_config.checkpoint_iters = 100     # This is a change
trainer = Trainer(train_config, model, train_dataset)

running on device cpu


In [24]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

iter_dt 0.00ms; iter 0: train loss 1.93167
iter_dt 39.91ms; iter 100: train loss 0.70072
iter_dt 35.40ms; iter 200: train loss 0.31527
iter_dt 31.90ms; iter 300: train loss 0.19964
iter_dt 42.26ms; iter 400: train loss 0.16451
iter_dt 52.50ms; iter 500: train loss 0.13598
iter_dt 47.99ms; iter 600: train loss 0.13150
iter_dt 43.97ms; iter 700: train loss 0.10623
iter_dt 42.96ms; iter 800: train loss 0.11184
iter_dt 38.27ms; iter 900: train loss 0.09307


In [35]:
from transformers import Trainer
from transformers import TrainingArguments

args = TrainingArguments(
    "dylanskinner65/mingpt_dylan",
    learning_rate=2e-5,
    evaluation_strategy='steps',
    eval_steps=100,
    weight_decay=0.1,
    adam_beta1=0.9,
    adam_beta2=0.95,
    push_to_hub=True,
)

trainer = Trainer(model, args, train_dataset=train_dataset, eval_dataset=test_dataset)
trainer.train()
trainer.push_to_hub("End of training")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


/Users/dylanskinner/opt/anaconda3/envs/acme1/lib/python3.7/site-packages/huggingface_hub/repository.py:708: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  FutureWarning,


HfHubHTTPError: <class 'requests.exceptions.HTTPError'> (Request ID: Root=1-654aad4c-15d094172fcba6040b9e14dc)

You don't have the rights to create a model under this namespace - You don't have the rights to create a model under this namespace

In [16]:
model.eval();

In [17]:
def eval_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        # isolate the input pattern alone
        inp = x[:, :n]
        sol = y[:, -n:]
        # let the model sample the rest of the sequence
        cat = model.generate(inp, n, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, n:] # isolate the filled in sequence
        # compare the predicted sequence to the true sequence
        correct = (sol == sol_candidate).all(1).cpu() # Software 1.0 vs. Software 2.0 fight RIGHT on this line haha
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            if not correct[i] and mistakes_printed_already < 3: # only print up to 5 mistakes to get a sense
                mistakes_printed_already += 1
                print("GPT claims that %s sorted is %s but gt is %s" % (inp[i].tolist(), sol_candidate[i].tolist(), sol[i].tolist()))
        if max_batches is not None and b+1 >= max_batches:
            break
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_split(trainer, 'train', max_batches=50)
    test_score  = eval_split(trainer, 'test',  max_batches=50)

GPT claims that [4, 4, 2, 4, 5, 5, 5, 5, 4, 1] sorted is [1, 2, 4, 4, 4, 4, 4, 5, 5, 5] but gt is [1, 2, 4, 4, 4, 4, 5, 5, 5, 5]
GPT claims that [5, 5, 4, 4, 4, 5, 4, 1, 5, 3] sorted is [1, 3, 4, 4, 4, 4, 4, 5, 5, 5] but gt is [1, 3, 4, 4, 4, 4, 5, 5, 5, 5]
GPT claims that [0, 4, 6, 5, 4, 4, 5, 6, 5, 5] sorted is [0, 4, 4, 4, 4, 5, 5, 5, 6, 6] but gt is [0, 4, 4, 4, 5, 5, 5, 5, 6, 6]
train final score: 4953/5000 = 99.06% correct
GPT claims that [0, 1, 0, 0, 5, 0, 0, 0, 4, 6] sorted is [0, 0, 0, 0, 0, 1, 3, 4, 5, 6] but gt is [0, 0, 0, 0, 0, 0, 1, 4, 5, 6]
GPT claims that [6, 6, 5, 5, 4, 4, 1, 4, 3, 3] sorted is [1, 3, 3, 4, 4, 4, 4, 5, 6, 6] but gt is [1, 3, 3, 4, 4, 4, 5, 5, 6, 6]
GPT claims that [3, 5, 4, 4, 5, 2, 5, 6, 4, 3] sorted is [2, 3, 3, 4, 4, 4, 4, 5, 5, 6] but gt is [2, 3, 3, 4, 4, 4, 5, 5, 5, 6]
test final score: 4943/5000 = 98.86% correct


In [10]:
# let's run a random given sequence through the model as well
n = train_dataset.length # naugy direct access shrug
inp = torch.tensor([[0, 0, 2, 1, 0, 1]], dtype=torch.long).to(trainer.device)
assert inp[0].nelement() == n
with torch.no_grad():
    cat = model.generate(inp, n, do_sample=False)
sol = torch.sort(inp[0])[0]
sol_candidate = cat[:, n:]
print('input sequence  :', inp.tolist())
print('predicted sorted:', sol_candidate.tolist())
print('gt sort         :', sol.tolist())
print('matches         :', bool((sol == sol_candidate).all()))

input sequence  : [[0, 0, 2, 1, 0, 1]]
predicted sorted: [[0, 0, 0, 1, 1, 2]]
gt sort         : [0, 0, 0, 1, 1, 2]
matches         : True
